In [1]:
import numpy as np
from utils import *
import tensorflow as tf
from sklearn.cross_validation import train_test_split
import time
from tqdm import tqdm
import random

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
maxlen = 100
kernel_size = 3
batch_size = 32
embedded_size = 128

In [3]:
trainset = sklearn.datasets.load_files(container_path = 'data', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [4]:
concat = ' '.join(trainset.data).split()
vocabulary_size = len(list(set(concat)))
data, count, dictionary, rev_dictionary = build_dataset(concat, vocabulary_size)
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [544, 2452, 3313, 13012, 36, 7203, 217, 150, 19, 3849] ['rock', 'destined', '21st', 'centurys', 'new', 'conan', 'hes', 'going', 'make', 'splash']


In [5]:
idx_trainset = []
for text in trainset.data:
    idx = []
    for t in text.split():
        try:
            idx.append(dictionary[t])
        except:
            pass
    idx_trainset.append(idx)

In [6]:
X = tf.keras.preprocessing.sequence.pad_sequences(idx_trainset, maxlen)
X.shape

(10662, 100)

In [7]:
train_X, test_X, train_Y, test_Y = train_test_split(X, 
                                                    trainset.target,
                                                    test_size = 0.2)

In [8]:
def gated_linear_unit(x, d_rate):
    c = tf.layers.conv1d(inputs = x,
                         filters = 2*embedded_size,
                         kernel_size = kernel_size,
                         dilation_rate = d_rate,
                         padding = 'same')
    c1, c2 = tf.split(c, 2, -1)
    x += c1 * tf.sigmoid(c2)
    return x

class Model:
    def __init__(self, embedded_size, dict_size, dimension_output, learning_rate):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None])
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        
        for d_rate in [1, 2, 4]:
            encoder_embedded = gated_linear_unit(encoder_embedded, d_rate)
    
        encoder_embedded = tf.reduce_max(encoder_embedded, 1)
        encoder_embedded = tf.layers.flatten(encoder_embedded)
        forward = tf.layers.dense(encoder_embedded, embedded_size, tf.nn.relu)
        self.logits = tf.layers.dense(forward, dimension_output)
        self.cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=self.logits,
            labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1,output_type=tf.int32), self.Y)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [9]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(embedded_size,vocabulary_size+4,2,1e-3)
sess.run(tf.global_variables_initializer())

In [10]:
EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : train_X[i:i+batch_size], model.Y : train_Y[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : test_X[i:i+batch_size], model.Y : test_Y[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

epoch: 0, pass acc: 0.000000, current acc: 0.614583
time taken: 5.76035475730896
epoch: 0, training loss: 0.682301, training acc: 0.571429, valid loss: 0.654988, valid acc: 0.614583

epoch: 1, pass acc: 0.614583, current acc: 0.648674
time taken: 3.361931800842285
epoch: 1, training loss: 0.344546, training acc: 0.862195, valid loss: 0.779541, valid acc: 0.648674

epoch: 2, pass acc: 0.648674, current acc: 0.678030
time taken: 3.365324020385742
epoch: 2, training loss: 0.085713, training acc: 0.970630, valid loss: 0.933929, valid acc: 0.678030

epoch: 3, pass acc: 0.678030, current acc: 0.681818
time taken: 3.3665685653686523
epoch: 3, training loss: 0.057237, training acc: 0.977679, valid loss: 1.200015, valid acc: 0.681818

time taken: 3.348708152770996
epoch: 4, training loss: 0.075732, training acc: 0.970042, valid loss: 1.340374, valid acc: 0.668561

time taken: 3.3564889430999756
epoch: 5, training loss: 0.035592, training acc: 0.986725, valid loss: 1.341547, valid acc: 0.676610


In [11]:
logits = sess.run(model.logits, feed_dict={model.X:test_X})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

             precision    recall  f1-score   support

   negative       0.70      0.60      0.65      1065
   positive       0.65      0.74      0.69      1068

avg / total       0.67      0.67      0.67      2133

